# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 06**: Writing Data in Neo4j

**Date**: October 5 2025

**Student Name**: Andre Jair Sanchez Contreras

**Professor**: Pablo Camarillo Ramirez

# Dataset description

In [ ]:
In this dataset 

Link:https://www.kaggle.com/datasets/zahidmughal2343/amazon-sales-2025?select=amazon_sales_data+2025.csv

# Data ingestion

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

USE_LOCAL = True  

neo4j_pkg = "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3"

if USE_LOCAL:
    
    spark = (SparkSession.builder
        .appName("Amazon → Neo4j (LOCAL)")
        .master("local[*]")                         
        .config("spark.jars.packages", neo4j_pkg)   
        .config("spark.ui.port", "4040")
        .getOrCreate())
else:
    
    spark = (SparkSession.builder
        .appName("Amazon → Neo4j (CLUSTER)")
        .master("spark://spark-master:7077")
        .config("spark.jars.packages", neo4j_pkg)
        .config("spark.ui.port", "4040")
        .getOrCreate())

sc = spark.sparkContext
sc.setLogLevel("ERROR")
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [ ]:
# Build schema
# Import your module

In [ ]:
from pcamarillor.spark_utils import SparkUtils

In [ ]:
from pyspark.sql import functions as F


csv_path = "/opt/spark/work-dir/data/amazon/amazon_sales_data_2025.csv"

df_raw = (spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(csv_path))

df = (df_raw
    .withColumnRenamed("Order ID", "order_id")
    .withColumn("date",          F.to_date("Date"))
    .withColumnRenamed("Product",           "product")
    .withColumnRenamed("Category",          "category")
    .withColumn("price",         F.col("Price").cast("double"))
    .withColumn("quantity",      F.col("Quantity").cast("int"))
    .withColumn("total_sales",   F.col("Total Sales").cast("double"))
    .withColumnRenamed("Customer Name",     "customer_name")
    .withColumnRenamed("Customer Location", "city")
    .withColumnRenamed("Payment Method",    "payment")
    .withColumnRenamed("Status",            "status")
    .select("order_id","date","product","category","price","quantity",
            "total_sales","customer_name","city","payment","status")
)



df.show(5, truncate=False)
df.printSchema()


# Transformations

In [ ]:
# Add the code for your transformations to create nodes and edges DataFrames HERE

# Writing Data in Neo4j

In [ ]:
# Add the code to write a graph from PySpark's DataFrames to Neo4j

# Read and Query Graphs with PySpark

In [ ]:
# Add the code to read a data frame from Neo4J and run a simple query to verify 


In [ ]:
sc.stop()